# **Módulo 3 - Manipulando dados com Spark - Parte I**

## **Importando bibliotecas do Spark**

In [ ]:
%%bash

# Instal Java
apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

In [ ]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

**OBS :** A preparação do ambiente não é igual ao da aula porque estou fazendo 
o exercício no Google Colab. 

In [ ]:
df = spark.createDataFrame([('Fulano','1'),
                            ('Ciclano','2')], 
                           schema='nome STRING, id STRING')

In [ ]:
df.show()

+-------+---+
|   nome| id|
+-------+---+
| Fulano|  1|
|Ciclano|  2|
+-------+---+



## **Acessando os Tipos do Spark**

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import col

In [ ]:
int_type = IntegerType()

In [ ]:
array_type = ArrayType(IntegerType())

In [ ]:
array_type

ArrayType(IntegerType,true)

## **Convertendo os tipos de Colunas**

In [ ]:
df.dtypes

[('nome', 'string'), ('id', 'string')]

In [ ]:
df.select('nome',col('id').cast(IntegerType()))

DataFrame[nome: string, id: int]

In [ ]:
df.show()

+-------+---+
|   nome| id|
+-------+---+
| Fulano|  1|
|Ciclano|  2|
+-------+---+



In [ ]:
df.select('nome',col('id').cast('int'))

DataFrame[nome: string, id: int]

In [ ]:
df.show()

+-------+---+
|   nome| id|
+-------+---+
| Fulano|  1|
|Ciclano|  2|
+-------+---+



## **Schema e Criação de DataFrames**

Um schema no Spark é uma especificação de tipos de colunas de um DataFrame. Eles são usadaos na leitura de dados externos e criação de DataFrames, e podem
ser passados diretamente no Spark ou podem ser inferidos. Passar um schema na 
leitura traz benefícios interessantes como : 

  - Evita que o Spark faça inferência de tipos, o que é custoso e demorado 
  dependendo do tamanho do arquivo, laém de propenso a erros; 
  - Permite que usuário identifique erros nos dados logo na leitura, caso 
  não sigam o schema especificado. 

In [ ]:
df = spark.createDataFrame([('Fulano'  ,1),
                            ('Ciclano' ,2),
                            ('Beltrano',3),
                            ('Deltrano',4)], 
                            schema=['nome','id'])

In [ ]:
df.show()

+--------+---+
|    nome| id|
+--------+---+
|  Fulano|  1|
| Ciclano|  2|
|Beltrano|  3|
|Deltrano|  4|
+--------+---+



In [ ]:
df.dtypes

[('nome', 'string'), ('id', 'bigint')]

## **Criando schemas programaticamente**

In [ ]:
schema = \
  StructType([
    StructField('nome',StringType()),
    StructField('id'  ,IntegerType())
  ])

In [ ]:
df = spark.createDataFrame([('Fulano'  ,1),
                            ('Ciclano' ,2),
                            ('Beltrano',3),
                            ('Deltrano',4)], 
                            schema=schema)

In [ ]:
df.show()

+--------+---+
|    nome| id|
+--------+---+
|  Fulano|  1|
| Ciclano|  2|
|Beltrano|  3|
|Deltrano|  4|
+--------+---+



In [ ]:
df.dtypes

[('nome', 'string'), ('id', 'int')]

## **Criando schemas com DDL**

In [ ]:
schema = 'nome STRING, id INT' 

In [ ]:
df = spark.createDataFrame([('Fulano'  ,1),
                            ('Ciclano' ,2),
                            ('Beltrano',3),
                            ('Deltrano',4)], 
                            schema=schema)

In [ ]:
df.show()

+--------+---+
|    nome| id|
+--------+---+
|  Fulano|  1|
| Ciclano|  2|
|Beltrano|  3|
|Deltrano|  4|
+--------+---+



In [ ]:
df.dtypes

[('nome', 'string'), ('id', 'int')]

## **Criando DataFrames**

In [ ]:
data = [('Fulano'  ,1),
        ('Ciclano' ,2),
        ('Beltrano',3),
        ('Deltrano',4)]

In [ ]:
schema = 'nome STRING, id INT' 

In [ ]:
df = spark.createDataFrame(data,schema=schema)

In [ ]:
df.dtypes

[('nome', 'string'), ('id', 'int')]

In [ ]:
df.printSchema()

root
 |-- nome: string (nullable = true)
 |-- id: integer (nullable = true)



In [ ]:
spark.range(100).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+
only showing top 20 rows



## **Leitura e Escrita de Dados**

In [ ]:
link_tab = '/content/drive/MyDrive/igti_bootcamps/eng_dados_cloud/mod3/tab_cnae.csv'

### **DataFrameReader**

      `spark.read.format(format).option(args).load(file_path)`

### **DataFrameWriter**

      `spark.write.format(format).option(args).load(file_path)`

### **Lendo e Escrevendo CSV**
Opções mais comuns : 
  - header
  - inferSchema
  - sep
  - encoding

In [ ]:
#df = spark.read.format('csv').load(link_tab)
df = spark.read.csv(link_tab,sep=',',header=True)

In [ ]:
df.limit(5).show()

+------+--------------------+---------+-----------+
|  CNAE|           DESCRIÇÃO|CÓD.SETOR| NOME SETOR|
+------+--------------------+---------+-----------+
|111301|    Cultivo de arroz|        1|AGRICULTURA|
|111302|    Cultivo de milho|        1|AGRICULTURA|
|111303|    Cultivo de trigo|        1|AGRICULTURA|
|111399|Cultivo de outros...|        1|AGRICULTURA|
|112101|Cultivo de algodã...|        1|AGRICULTURA|
+------+--------------------+---------+-----------+



## **Definindo o schema**

In [ ]:
schema = 'cod_cnae STRING, descricao STRING, cod_setor INT, nome_setor STRING '

In [ ]:
df = spark.read.csv(link_tab,sep=',',header=True, schema=schema)

In [ ]:
df.dtypes

[('cod_cnae', 'string'),
 ('descricao', 'string'),
 ('cod_setor', 'int'),
 ('nome_setor', 'string')]

outra forma de fazer seria

In [ ]:
df = (
    spark.read
    .format('csv')
    .option('header','true')
    .option('sep',',')
    .schema(schema)
    .load(link_tab)
    )

df.limit(5).show()

+--------+--------------------+---------+-----------+
|cod_cnae|           descricao|cod_setor| nome_setor|
+--------+--------------------+---------+-----------+
|  111301|    Cultivo de arroz|        1|AGRICULTURA|
|  111302|    Cultivo de milho|        1|AGRICULTURA|
|  111303|    Cultivo de trigo|        1|AGRICULTURA|
|  111399|Cultivo de outros...|        1|AGRICULTURA|
|  112101|Cultivo de algodã...|        1|AGRICULTURA|
+--------+--------------------+---------+-----------+



In [ ]:
df = (
    spark.read
    .format('csv')
    .options(header=True,sep=',')  
    .schema(schema)
    .load(link_tab)
)
df.show(5)

+--------+--------------------+---------+-----------+
|cod_cnae|           descricao|cod_setor| nome_setor|
+--------+--------------------+---------+-----------+
|  111301|    Cultivo de arroz|        1|AGRICULTURA|
|  111302|    Cultivo de milho|        1|AGRICULTURA|
|  111303|    Cultivo de trigo|        1|AGRICULTURA|
|  111399|Cultivo de outros...|        1|AGRICULTURA|
|  112101|Cultivo de algodã...|        1|AGRICULTURA|
+--------+--------------------+---------+-----------+
only showing top 5 rows



**OBS :** Utilizando o método "options" podemos parametrizar melhor nossa função
 usando um dicionário

In [ ]:
options_dict = {
    'sep' : ',' , 
    'header' : 'True'
}

df = (
      spark.read
    .format('csv')
    .options(**options_dict)  
    .schema(schema)
    .load(link_tab)
)
df.show(5)

+--------+--------------------+---------+-----------+
|cod_cnae|           descricao|cod_setor| nome_setor|
+--------+--------------------+---------+-----------+
|  111301|    Cultivo de arroz|        1|AGRICULTURA|
|  111302|    Cultivo de milho|        1|AGRICULTURA|
|  111303|    Cultivo de trigo|        1|AGRICULTURA|
|  111399|Cultivo de outros...|        1|AGRICULTURA|
|  112101|Cultivo de algodã...|        1|AGRICULTURA|
+--------+--------------------+---------+-----------+
only showing top 5 rows



In [ ]:
df.printSchema()

root
 |-- cod_cnae: string (nullable = true)
 |-- descricao: string (nullable = true)
 |-- cod_setor: integer (nullable = true)
 |-- nome_setor: string (nullable = true)



In [ ]:
schema = 'cod_cnae INT, descricao STRING, cod_setor INT, nome_setor STRING '

In [ ]:
options_dict = {
    'sep' : ',' , 
    'header' : 'True'
}

df = (
      spark.read
    .format('csv')
    .options(**options_dict)  
    .schema(schema)
    .load(link_tab)
)
df.show(5)

+--------+--------------------+---------+-----------+
|cod_cnae|           descricao|cod_setor| nome_setor|
+--------+--------------------+---------+-----------+
|  111301|    Cultivo de arroz|        1|AGRICULTURA|
|  111302|    Cultivo de milho|        1|AGRICULTURA|
|  111303|    Cultivo de trigo|        1|AGRICULTURA|
|  111399|Cultivo de outros...|        1|AGRICULTURA|
|  112101|Cultivo de algodã...|        1|AGRICULTURA|
+--------+--------------------+---------+-----------+
only showing top 5 rows



In [ ]:
df.printSchema()

root
 |-- cod_cnae: integer (nullable = true)
 |-- descricao: string (nullable = true)
 |-- cod_setor: integer (nullable = true)
 |-- nome_setor: string (nullable = true)



In [ ]:
path_save = '/content/drive/MyDrive/igti_bootcamps/eng_dados_cloud/mod3/'

In [ ]:
#df.write.format('csv').save(path_save + 'df_cnae_teste',header=True)

AnalysisException: ignored

In [ ]:
spark.read.format('csv').load(path_save + 'df_cnae_teste',header=True).printSchema()

root
 |-- cod_cnae: string (nullable = true)
 |-- descricao: string (nullable = true)
 |-- cod_setor: string (nullable = true)
 |-- nome_setor: string (nullable = true)



In [ ]:
df.toPandas().to_csv(
    path_save + 'df_cnae_teste.csv',
    index=False, header=True
    )

# **Aula 3.4.2 -** Leitura e escrita de dados - Parte II

## **Lendo e Escrevendo JSON**

In [ ]:
df.write.format('json') \
  .save(path_save + 'df_cnae_teste.json')

In [ ]:
df_json = spark.read.format('json') \
  .load(path_save + 'df_cnae_teste.json')

In [ ]:
df_json.show()

+--------+---------+--------------------+-----------+
|cod_cnae|cod_setor|           descricao| nome_setor|
+--------+---------+--------------------+-----------+
|  111301|        1|    Cultivo de arroz|AGRICULTURA|
|  111302|        1|    Cultivo de milho|AGRICULTURA|
|  111303|        1|    Cultivo de trigo|AGRICULTURA|
|  111399|        1|Cultivo de outros...|AGRICULTURA|
|  112101|        1|Cultivo de algodã...|AGRICULTURA|
|  112102|        1|     Cultivo de juta|AGRICULTURA|
|  112199|        1|Cultivo de outras...|AGRICULTURA|
|  113000|        1|Cultivo de cana-d...|AGRICULTURA|
|  114800|        1|     Cultivo de fumo|AGRICULTURA|
|  115600|        1|     Cultivo de soja|AGRICULTURA|
|  116401|        1| Cultivo de amendoim|AGRICULTURA|
|  116402|        1| Cultivo de girassol|AGRICULTURA|
|  116403|        1|   Cultivo de mamona|AGRICULTURA|
|  116499|        1|Cultivo de outras...|AGRICULTURA|
|  119901|        1|  Cultivo de abacaxi|AGRICULTURA|
|  119902|        1|     Cul

In [ ]:
df_json.printSchema()

root
 |-- cod_cnae: long (nullable = true)
 |-- cod_setor: long (nullable = true)
 |-- descricao: string (nullable = true)
 |-- nome_setor: string (nullable = true)



## **Lendo e Escrevendo ORC**



In [ ]:
df.write.format('orc').save(path_save + 'df_cnae_teste.orc')

In [ ]:
df_orc = spark.read.format('orc').load(path_save + 'df_cnae_teste.orc')

In [ ]:
df_orc.show(5)

+--------+--------------------+---------+-----------+
|cod_cnae|           descricao|cod_setor| nome_setor|
+--------+--------------------+---------+-----------+
|  111301|    Cultivo de arroz|        1|AGRICULTURA|
|  111302|    Cultivo de milho|        1|AGRICULTURA|
|  111303|    Cultivo de trigo|        1|AGRICULTURA|
|  111399|Cultivo de outros...|        1|AGRICULTURA|
|  112101|Cultivo de algodã...|        1|AGRICULTURA|
+--------+--------------------+---------+-----------+
only showing top 5 rows



In [ ]:
df_orc.printSchema()

root
 |-- cod_cnae: integer (nullable = true)
 |-- descricao: string (nullable = true)
 |-- cod_setor: integer (nullable = true)
 |-- nome_setor: string (nullable = true)



## **Lendo e Escrevendo Parquet**

  - Armazenamento colunar, em contraste com o CSV, que armazena baseado nas linhas. Assim, quando uma query é realizada é possível ignorar os dados não relevantes de maneira rápida e fácil, resultando em operações bem mais eficientes; 
  - Preservação de metadados, incluindo os tipos das colunas, o que garante eficiência e praticidade na escrita e leitura (não é necessário especificar schemas para arquivos parquet);
  - Suporte a dados estruturados de forma aninhada, como listas;
  - Otimizado para processar dados particionados com volume na casa dos gigabytes para cada arquivo;
  - Compressão de dados na escrita, de forma a ocupar menos espaço;
  - Integração com ferramentas como AWS athena, Amazon Redshift Spectrum, Google Big Query e Google Dataproc. 

In [ ]:
df.write.format('parquet').save(path_save + 'df_cnae_teste.pq')

In [ ]:
df_parquet = spark.read.format('parquet').load(path_save + 'df_cnae_teste.pq')

In [ ]:
df_parquet.show(5)

+--------+--------------------+---------+-----------+
|cod_cnae|           descricao|cod_setor| nome_setor|
+--------+--------------------+---------+-----------+
|  111301|    Cultivo de arroz|        1|AGRICULTURA|
|  111302|    Cultivo de milho|        1|AGRICULTURA|
|  111303|    Cultivo de trigo|        1|AGRICULTURA|
|  111399|Cultivo de outros...|        1|AGRICULTURA|
|  112101|Cultivo de algodã...|        1|AGRICULTURA|
+--------+--------------------+---------+-----------+
only showing top 5 rows



In [ ]:
df_parquet.printSchema()

root
 |-- cod_cnae: integer (nullable = true)
 |-- descricao: string (nullable = true)
 |-- cod_setor: integer (nullable = true)
 |-- nome_setor: string (nullable = true)



mode : 
  
  - append : arquivos empilhados aos já existentes
  - ignore : retorna um erro silencioso
  - overwrite : sobrescreve os dados já exsitentes
  -  error (default) : retorne erro se já existem dados

In [ ]:
df.write.format('parquet').save(path_save + 'df_cnae_teste.pq', 
                                mode='overwrite'
                                )

ou

In [ ]:
df.write.format('parquet').mode('overwrite').save(path_save + 'df_cnae_teste.pq')

# **Aula 3.5.1 -** Operações básicas com DataFrames - Parte I

## **Manipulação dos Dados**

In [ ]:
imdb_path = '/content/drive/MyDrive/igti_bootcamps/eng_dados_cloud/mod3/title_basics.csv'

In [ ]:
options_dict = {
    'sep' : '\t' , 
    'header' : 'True'
}

df_titles = (
      spark.read
    .format('csv')
    .options(**options_dict)  
    .load(imdb_path)
)

df_titles.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

## **Colunas e Expressões**

As colunas são a principal unidade de manipulação de dados do Spark

In [ ]:
from pyspark.sql.functions import col,round

In [ ]:
(
    df_titles.select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeHours',col('runTimeMinutes').cast('int')/60)
    .show(5)
)

+---------+--------------------+--------------+--------------------+
|   tconst|        primaryTitle|runtimeMinutes|        runtimeHours|
+---------+--------------------+--------------+--------------------+
|tt0000001|          Carmencita|             1|0.016666666666666666|
|tt0000002|Le clown et ses c...|             5| 0.08333333333333333|
|tt0000003|      Pauvre Pierrot|             4| 0.06666666666666667|
|tt0000004|         Un bon bock|            12|                 0.2|
|tt0000005|    Blacksmith Scene|             1|0.016666666666666666|
+---------+--------------------+--------------+--------------------+
only showing top 5 rows



In [ ]:
(
    df_titles.select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeHours',round(col('runTimeMinutes').cast('int')/60,3))
    .show(5)
)

+---------+--------------------+--------------+------------+
|   tconst|        primaryTitle|runtimeMinutes|runtimeHours|
+---------+--------------------+--------------+------------+
|tt0000001|          Carmencita|             1|       0.017|
|tt0000002|Le clown et ses c...|             5|       0.083|
|tt0000003|      Pauvre Pierrot|             4|       0.067|
|tt0000004|         Un bon bock|            12|         0.2|
|tt0000005|    Blacksmith Scene|             1|       0.017|
+---------+--------------------+--------------+------------+
only showing top 5 rows



In [ ]:
df_titles['runtimeMinutes']

Column<'runtimeMinutes'>

Forma "pandas" de selecionar

  1. df.coluna
  2. df['coluna']

In [ ]:
(
    df_titles.select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeHours',round(col('runTimeMinutes').cast('int')/60,3))
    .show(5)
)

+---------+--------------------+--------------+------------+
|   tconst|        primaryTitle|runtimeMinutes|runtimeHours|
+---------+--------------------+--------------+------------+
|tt0000001|          Carmencita|             1|       0.017|
|tt0000002|Le clown et ses c...|             5|       0.083|
|tt0000003|      Pauvre Pierrot|             4|       0.067|
|tt0000004|         Un bon bock|            12|         0.2|
|tt0000005|    Blacksmith Scene|             1|       0.017|
+---------+--------------------+--------------+------------+
only showing top 5 rows



In [ ]:
(
    df_titles.select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeHours',df_titles['runtimeMinutes'].cast('int')/60)
    .withColumn('hours_plus2' ,df_titles['runtimeHours'] + 2)
    .show(5)
)

AnalysisException: ignored

In [ ]:
(
    df_titles.select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeHours',col('runtimeMinutes').cast('int')/60)
    .withColumn('hours_plus2' ,col('runtimeHours') + 2)
    .show(5)
)

+---------+--------------------+--------------+--------------------+------------------+
|   tconst|        primaryTitle|runtimeMinutes|        runtimeHours|       hours_plus2|
+---------+--------------------+--------------+--------------------+------------------+
|tt0000001|          Carmencita|             1|0.016666666666666666|2.0166666666666666|
|tt0000002|Le clown et ses c...|             5| 0.08333333333333333|2.0833333333333335|
|tt0000003|      Pauvre Pierrot|             4| 0.06666666666666667| 2.066666666666667|
|tt0000004|         Un bon bock|            12|                 0.2|               2.2|
|tt0000005|    Blacksmith Scene|             1|0.016666666666666666|2.0166666666666666|
+---------+--------------------+--------------+--------------------+------------------+
only showing top 5 rows



### **Expressões**

In [ ]:
from pyspark.sql.functions import expr

In [ ]:
(
    df_titles.select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeHours',expr('cast(runtimeMinutes as INT)/60'))
    .show(5)
)

+---------+--------------------+--------------+--------------------+
|   tconst|        primaryTitle|runtimeMinutes|        runtimeHours|
+---------+--------------------+--------------+--------------------+
|tt0000001|          Carmencita|             1|0.016666666666666666|
|tt0000002|Le clown et ses c...|             5| 0.08333333333333333|
|tt0000003|      Pauvre Pierrot|             4| 0.06666666666666667|
|tt0000004|         Un bon bock|            12|                 0.2|
|tt0000005|    Blacksmith Scene|             1|0.016666666666666666|
+---------+--------------------+--------------+--------------------+
only showing top 5 rows



In [ ]:
(
    df_titles.select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeHours',expr('round(cast(runtimeMinutes as INT)/60,3)'))
    .show(5)
)

+---------+--------------------+--------------+------------+
|   tconst|        primaryTitle|runtimeMinutes|runtimeHours|
+---------+--------------------+--------------+------------+
|tt0000001|          Carmencita|             1|       0.017|
|tt0000002|Le clown et ses c...|             5|       0.083|
|tt0000003|      Pauvre Pierrot|             4|       0.067|
|tt0000004|         Un bon bock|            12|         0.2|
|tt0000005|    Blacksmith Scene|             1|       0.017|
+---------+--------------------+--------------+------------+
only showing top 5 rows



# **Aula 3.5.2 -** Operações básicas com DataFrames - Parte II

## **Manipulação dos Dados**

In [ ]:
df_titles.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [ ]:
df_titles.select('tconst','primaryTitle','genres').show(5)

+---------+--------------------+--------------------+
|   tconst|        primaryTitle|              genres|
+---------+--------------------+--------------------+
|tt0000001|          Carmencita|   Documentary,Short|
|tt0000002|Le clown et ses c...|     Animation,Short|
|tt0000003|      Pauvre Pierrot|Animation,Comedy,...|
|tt0000004|         Un bon bock|     Animation,Short|
|tt0000005|    Blacksmith Scene|        Comedy,Short|
+---------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
df_titles.columns

['tconst',
 'titleType',
 'primaryTitle',
 'originalTitle',
 'isAdult',
 'startYear',
 'endYear',
 'runtimeMinutes',
 'genres']

In [ ]:
select_cols = [c for c in df_titles.columns if c.find('Title') != -1]

In [ ]:
df_titles.select(select_cols).show(5)

+--------------------+--------------------+
|        primaryTitle|       originalTitle|
+--------------------+--------------------+
|          Carmencita|          Carmencita|
|Le clown et ses c...|Le clown et ses c...|
|      Pauvre Pierrot|      Pauvre Pierrot|
|         Un bon bock|         Un bon bock|
|    Blacksmith Scene|    Blacksmith Scene|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
cols = ['tconst','primaryTitle','genres']
df_titles.select(cols).show(10)

+---------+--------------------+--------------------+
|   tconst|        primaryTitle|              genres|
+---------+--------------------+--------------------+
|tt0000001|          Carmencita|   Documentary,Short|
|tt0000002|Le clown et ses c...|     Animation,Short|
|tt0000003|      Pauvre Pierrot|Animation,Comedy,...|
|tt0000004|         Un bon bock|     Animation,Short|
|tt0000005|    Blacksmith Scene|        Comedy,Short|
|tt0000006|   Chinese Opium Den|               Short|
|tt0000007|Corbett and Court...|         Short,Sport|
|tt0000008|Edison Kinetoscop...|   Documentary,Short|
|tt0000009|          Miss Jerry|       Romance,Short|
|tt0000010| Leaving the Factory|   Documentary,Short|
+---------+--------------------+--------------------+
only showing top 10 rows



In [ ]:
cols = ['primaryTitle','genres']
df_titles.select('tconst',*cols).show(5)

+---------+--------------------+--------------------+
|   tconst|        primaryTitle|              genres|
+---------+--------------------+--------------------+
|tt0000001|          Carmencita|   Documentary,Short|
|tt0000002|Le clown et ses c...|     Animation,Short|
|tt0000003|      Pauvre Pierrot|Animation,Comedy,...|
|tt0000004|         Un bon bock|     Animation,Short|
|tt0000005|    Blacksmith Scene|        Comedy,Short|
+---------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
df_titles.select('*').show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

**Observações :**

  - Podemos realizar operações sobre colunas selecionadas
  - A ordem em que as colunas são selecionadas é a ordem em que elas vão ser inseridas no DataFrame resultante.

In [ ]:
from pyspark.sql.functions import upper

df_titles.select('tconst',
                 'genres',
                 upper('primaryTitle').alias('primaryTitle')).show(5)

+---------+--------------------+--------------------+
|   tconst|              genres|        primaryTitle|
+---------+--------------------+--------------------+
|tt0000001|   Documentary,Short|          CARMENCITA|
|tt0000002|     Animation,Short|LE CLOWN ET SES C...|
|tt0000003|Animation,Comedy,...|      PAUVRE PIERROT|
|tt0000004|     Animation,Short|         UN BON BOCK|
|tt0000005|        Comedy,Short|    BLACKSMITH SCENE|
+---------+--------------------+--------------------+
only showing top 5 rows



**Selecionando valores distintos**

In [ ]:
df_titles.select('startYear').distinct().show()

+---------+
|startYear|
+---------+
|     1903|
|     1953|
|     1897|
|     1957|
|     1987|
|     1956|
|     1936|
|     2016|
|     2020|
|     2012|
|     1958|
|     1910|
|     1943|
|     1915|
|     1972|
|     1931|
|     1911|
|     1926|
|     1938|
|     1988|
+---------+
only showing top 20 rows



In [ ]:
df_titles.distinct().show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000033|    short|  Horse Trick Riders|          La voltige|      0|     1895|     \N|             1|Comedy,Documentar...|
|tt0000132|    short|          Card Party|Une partie de cartes|      0|     1896|     \N|             1|     Biography,Short|
|tt0214902|    short|        The Magician|         Le magicien|      0|     1898|     \N|             1|Fantasy,Horror,Short|
|tt0225248|    short|Canada Vignettes:...|Canada Vignettes:...|      0|     1979|     \N|             1|     Animation,Short|
|tt0245776|    short|       The Biter Bit|       The Biter Bit|      0|     1899|     \N|             1|        Comedy

In [ ]:
df_titles.count()

8328316

In [ ]:
df_titles.distinct().explain('formatted')

== Physical Plan ==
AdaptiveSparkPlan (5)
+- HashAggregate (4)
   +- Exchange (3)
      +- HashAggregate (2)
         +- Scan csv  (1)


(1) Scan csv 
Output [9]: [tconst#530, titleType#531, primaryTitle#532, originalTitle#533, isAdult#534, startYear#535, endYear#536, runtimeMinutes#537, genres#538]
Batched: false
Location: InMemoryFileIndex [file:/content/drive/MyDrive/igti_bootcamps/eng_dados_cloud/mod3/title_basics.csv]
ReadSchema: struct<tconst:string,titleType:string,primaryTitle:string,originalTitle:string,isAdult:string,startYear:string,endYear:string,runtimeMinutes:string,genres:string>

(2) HashAggregate
Input [9]: [tconst#530, titleType#531, primaryTitle#532, originalTitle#533, isAdult#534, startYear#535, endYear#536, runtimeMinutes#537, genres#538]
Keys [9]: [runtimeMinutes#537, tconst#530, titleType#531, originalTitle#533, startYear#535, endYear#536, primaryTitle#532, isAdult#534, genres#538]
Functions: []
Aggregate Attributes: []
Results [9]: [runtimeMinutes#537, tconst#53

**Filtros**

Operadores lógicos : 
   - e   : &
   - ou  : |
   - não : ~

Para fazer o filtro, pode ser usado tanto a função `filter()` como `where()` 

**Filtros com uma condição**

In [ ]:
(
    df_titles.filter(col('titleType') == 'movie').show(5)
)

+---------+---------+------------------+------------------+-------+---------+-------+--------------+------+
|   tconst|titleType|      primaryTitle|     originalTitle|isAdult|startYear|endYear|runtimeMinutes|genres|
+---------+---------+------------------+------------------+-------+---------+-------+--------------+------+
|tt0000502|    movie|          Bohemios|          Bohemios|      0|     1905|     \N|           100|    \N|
|tt0000591|    movie|  The Prodigal Son| L'enfant prodigue|      0|     1907|     \N|            90| Drama|
|tt0000615|    movie|Robbery Under Arms|Robbery Under Arms|      0|     1907|     \N|            \N| Drama|
|tt0000630|    movie|            Hamlet|            Amleto|      0|     1908|     \N|            \N| Drama|
|tt0000675|    movie|       Don Quijote|       Don Quijote|      0|     1908|     \N|            \N| Drama|
+---------+---------+------------------+------------------+-------+---------+-------+--------------+------+
only showing top 5 rows



In [ ]:
(
    df_titles.where(col('titleType') == 'movie').show()
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|           genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|tt0000502|    movie|            Bohemios|            Bohemios|      0|     1905|     \N|           100|               \N|
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|            Drama|
|tt0000615|    movie|  Robbery Under Arms|  Robbery Under Arms|      0|     1907|     \N|            \N|            Drama|
|tt0000630|    movie|              Hamlet|              Amleto|      0|     1908|     \N|            \N|            Drama|
|tt0000675|    movie|         Don Quijote|         Don Quijote|      0|     1908|     \N|            \N|            Drama|
|tt0000679|    m

In [ ]:
(
    df_titles.where(col('titleType') == 'movie').count()
)

591977

In [ ]:
(
    df_titles.filter((col('titleType') == 'movie') & (col('runTimeMinutes') <= 90)).show()
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0001184|    movie|Don Juan de Serra...|Don Juan de Serra...|      0|     1910|     \N|            58|     Adventure,Drama|
|tt0001258|    movie|The White Slave T...|Den hvide slaveha...|      0|     1910|     \N|            45|               Drama|
|tt0001285|    movie|   The Life of Moses|   The Life of Moses|      0|     1909|     \N|            50|Biography,Drama,F...|
|tt0001498|    movie|The Battle of Tra...|The Battle of Tra...|      0|     1911|     \N|            51|              

In [ ]:
(
    df_titles.filter(((col('titleType') == 'movie') | (col('titleType') == 'tvSeries')) & (col('runtimeMinutes') <= 90)).show()
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0001184|    movie|Don Juan de Serra...|Don Juan de Serra...|      0|     1910|     \N|            58|     Adventure,Drama|
|tt0001258|    movie|The White Slave T...|Den hvide slaveha...|      0|     1910|     \N|            45|               Drama|
|tt0001285|    movie|   The Life of Moses|   The Life of Moses|      0|     1909|     \N|            50|Biography,Drama,F...|
|tt0001498|    movie|The Battle of Tra...|The Battle of Tra...|      0|     1911|     \N|            51|              

In [ ]:
(
df_titles.filter( ((col('titleType') == 'movie') & (col('titleType') == 'tvSeries') & (col('runtimeMinutes') <= 90)) ).count()
)

0

In [ ]:
(
    df_titles.filter(((col('titleType') == 'movie') | (col('titleType') == 'tvSeries')) & (col('runtimeMinutes') <= 90)).count()
)

296182

In [ ]:
df_titles.filter((col('titleType').isin('movie','tvSeries')) & (col('runtimeMinutes') <= 90)).show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0001184|    movie|Don Juan de Serra...|Don Juan de Serra...|      0|     1910|     \N|            58|     Adventure,Drama|
|tt0001258|    movie|The White Slave T...|Den hvide slaveha...|      0|     1910|     \N|            45|               Drama|
|tt0001285|    movie|   The Life of Moses|   The Life of Moses|      0|     1909|     \N|            50|Biography,Drama,F...|
|tt0001498|    movie|The Battle of Tra...|The Battle of Tra...|      0|     1911|     \N|            51|              

In [ ]:
(
df_titles
.filter(col('titleType').isin('movie','tvSeries'))
.filter(col('runtimeMinutes') <= 90)
.count()
)

296182

**Filtros utilizando expressões**

In [ ]:
(
    df_titles
    .filter('titleType == "movie"')
    .filter('runtimeMinutes <= 90')
    .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0001184|    movie|Don Juan de Serra...|Don Juan de Serra...|      0|     1910|     \N|            58|     Adventure,Drama|
|tt0001258|    movie|The White Slave T...|Den hvide slaveha...|      0|     1910|     \N|            45|               Drama|
|tt0001285|    movie|   The Life of Moses|   The Life of Moses|      0|     1909|     \N|            50|Biography,Drama,F...|
|tt0001498|    movie|The Battle of Tra...|The Battle of Tra...|      0|     1911|     \N|            51|              

In [ ]:
(
    df_titles
    .filter('titleType in ("movie","tvSeries") and runtimeMinutes <= 90')
    .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0001184|    movie|Don Juan de Serra...|Don Juan de Serra...|      0|     1910|     \N|            58|     Adventure,Drama|
|tt0001258|    movie|The White Slave T...|Den hvide slaveha...|      0|     1910|     \N|            45|               Drama|
|tt0001285|    movie|   The Life of Moses|   The Life of Moses|      0|     1909|     \N|            50|Biography,Drama,F...|
|tt0001498|    movie|The Battle of Tra...|The Battle of Tra...|      0|     1911|     \N|            51|              

**Observações**

Quando nos referimos às colunas por meio da função `col()`, temos acesso à diversos métodos das colunas que podem ser utilizados para auxiliar na filtragem do DataFrame. Alguns deles são :

  - `isin()` : checa se a coluna contém os valores listados na função.
  - `contains` : utilizado para verificar se uma coluna de texto contém algum padrão especificado (não aceita regex). Aceita uma outra coluna de text.
  - `like()` : utilizado para verificar se uma coluna de texto contém algum padrão especificado (não aceita regex). Funciona de forma similar so "LIKE" do SQL.
  - `rlike()` : utilizado para verificar se uma coluna de texto contém algum padrão especificado (**aceita regex**). Funciona de forma similar ao "RLIKE" do SQL
  - `startswith()` : utilizado para verificar se uma coluna de texto começa com algum padrão especificado (**aceita regex**).
  - `endswith()` :  utilizado para verificar se uma coluna de texto termina com algum padrão especificado (**aceita regex**).
  - `between()` : checa se os valores da coluna estão dentro do intervalo especificado. Os dois lados do intervalo são inclusivos
  - `isNull()` : retorna True se o valor da coluna é nulo
  - `isNotNull()` : retorna True se o valor da coluna é não nulo

Outros métodos úteis :

  - `alias()/name()` : usado para renomear as colunas em operações como select() e agg()
  - `astype()/cast()` : usado para mudar o tipo das colunas. Aceita tanto um
  string como um tipo especificado pelo módulo pyspark.sql.types
  - `substr()` : utilizado para cortar uma string com base em índices dos caracteres.  

In [ ]:
(
  df_titles
 .filter(col('primaryTitle').like('%Avengers%'))
 .filter(col('titleType') == 'movie')
 .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0034639|    movie|        The Avengers|   The Day Will Dawn|      0|     1942|     \N|            98|           Drama,War|
|tt0036194|    movie|The People's Aven...|   Narodnye mstiteli|      0|     1943|     \N|            55|     Documentary,War|
|tt0058651|    movie|  The Three Avengers| Gli invincibili tre|      0|     1964|     \N|           101|Action,Adventure,...|
|tt0069746|    movie|Avengers of the Reef|Avengers of the Reef|      0|     1973|     \N|            84|    Adventure,Family|
|tt0074513|    movie|The Shaolin Avengers|Fang Shi Yu yu Hu...|      0|     1976|     \N|            97|        Action

In [ ]:
(
    df_titles.
    withColumn('startYear',col('startYear').cast('int'))
    .filter(col('startYear').isNotNull())
    .show()
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

# **Aula 3.5.3 -** Operações básicas com DataFrames - Parte III

**Ordenando o DataFrame**

A ordenação do DataFrame pode ser feita utilizando funções `orderBy()` ou `sort()`. Algumas funções auxiliares importantes para serem usadas ao ordenar : 

  - `asc()`: ordena a coluna de forma ascendente (default)
  - `desc()` : ordena a coluna de forma descrescente
  - `asc_nulls_first()/desc_nulls_first()`: ordena a coluna de forma ascendente e descendente, respectivamente, mantendo os campos nulos primeiro
  - `asc_nulls_last()/desc_nulls_last()`: ordena a coluna de forma ascendente e
  decrescente, respectivamente, mantendo os campos nulos por último

In [ ]:
df_titles.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [ ]:
df_titles.orderBy(col('startYear').cast('int')).show()

+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+-------------+
|    tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|       genres|
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+-------------+
|tt11112784|    movie|   Perfect Strangers|   Perfect Strangers|      0|       \N|     \N|            \N| Comedy,Drama|
|tt11113108|tvEpisode|      Episode #1.759|      Episode #1.759|      0|       \N|     \N|            \N|Comedy,Family|
|tt11112532|    movie|                Agra|                Agra|      0|       \N|     \N|            \N|        Drama|
|tt11113062|tvEpisode|      Episode #1.741|      Episode #1.741|      0|       \N|     \N|            \N|Comedy,Family|
|tt11112858|    movie|Lizards: A Pop Opera|Lizards: A Pop Opera|      0|       \N|     \N|            \N|        Drama|
|tt11113066|tvEpisode|      Episode #1.7

In [ ]:
df_titles.orderBy('startYear','runtimeMinutes').filter('titleType == "movie"').show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt2210499|    movie|          Birmingham|          Birmingham|      0|     1896|     \N|            61|         Documentary|
|tt0229676|    movie|Reproduction of t...|Reproduction of t...|      0|     1897|     \N|            \N|Documentary,News,...|
|tt0138342|    movie|      O Campo Grande|      O Campo Grande|      0|     1898|     \N|            \N|         Documentary|
|tt0138349|    movie|O Carnaval em Lisboa|O Carnaval em Lisboa|      0|     1898|     \N|            \N|         Documentary|
|tt0138759|    movie|A Rua Augusta em ...|A Rua Augusta em ...|      0|     1898|     \N|            \N|         Docum

In [ ]:
from pyspark.sql.functions import desc

df_titles.orderBy('startYear',desc('runtimeMinutes')).filter('titleType == "movie"').show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt2210499|    movie|          Birmingham|          Birmingham|      0|     1896|     \N|            61|         Documentary|
|tt0229676|    movie|Reproduction of t...|Reproduction of t...|      0|     1897|     \N|            \N|Documentary,News,...|
|tt0236940|    movie|69th Regiment Pas...|69th Regiment Pas...|      0|     1898|     \N|            \N|         Documentary|
|tt0235357|    movie|Dressing Paper Dolls|Dressing Paper Dolls|      0|     1898|     \N|            \N|         Documentary|
|tt0138349|    movie|O Carnaval em Lisboa|O Carnaval em Lisboa|      0|     1898|     \N|            \N|         Docum

In [ ]:
df_titles.orderBy(desc('startYear')).filter('titleType == "movie"').show()

+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|    tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt11115868|    movie|        The Shinobis|        The Shinobis|      0|       \N|     \N|            \N|              Action|
|tt11116102|    movie|              Deeper|              Deeper|      0|       \N|     \N|            \N|             Romance|
|tt11116112|    movie|Untitled Elliott ...|Untitled Elliott ...|      0|       \N|     \N|            \N|                  \N|
|tt11112784|    movie|   Perfect Strangers|   Perfect Strangers|      0|       \N|     \N|            \N|        Comedy,Drama|
|tt11116440|    movie|               Shame|               Shame|      0|       \N|     \N|            \N|      

In [ ]:
from pyspark.sql.functions import desc

(
 df_titles.withColumn('startYear' , col('startYear').cast('int'))
 .orderBy('startYear')
 .filter('titleType == "movie"')
 .show()
)

+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|    tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt11116440|    movie|               Shame|               Shame|      0|     null|     \N|            \N|                  \N|
|tt11122910|    movie|       Durgo Rahasya|       Durgo Rahasya|      0|     null|     \N|            \N|      Drama,Thriller|
|tt11116516|    movie|        Spiritwalker|        Spiritwalker|      0|     null|     \N|            \N|      Fantasy,Horror|
|tt11112072|    movie|       Hell Can Wait|       Hell Can Wait|      0|     null|     \N|            \N|              Comedy|
|tt11116536|    movie|           3rd Floor|           3rd Floor|      0|     null|     \N|            \N|      

In [ ]:
from pyspark.sql.functions import asc_nulls_first

(
  df_titles
 .withColumn('startYear',col('startYear').cast('int'))
 .orderBy(asc_nulls_first('startYear'))
 .show(5)   
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|      genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+------------+
|tt0067098|tvEpisode|         Willi Forst|         Willi Forst|      0|     null|     \N|            55|          \N|
|tt0092975|tvEpisode|Erste Liebe und H...|Erste Liebe und H...|      0|     null|     \N|            \N| Documentary|
|tt0071158|tvEpisode|  The Arcata Promise|  The Arcata Promise|      0|     null|     \N|            \N|       Drama|
|tt0073399|    movie|Atlantic City Jac...|           The Money|      0|     null|     \N|            88|Action,Drama|
|tt0085677|tvEpisode|        High Country|        High Country|      0|     null|     \N|            \N|       Sport|
+---------+---------+--------------------+--------------

**Renomeando Colunas**

Para renomear colunas, é utilizada a função `withColumnRename()` , da seguinte forma

  `df.withColumnRenamed("nome_antigo","nome_novo")`

In [ ]:
(
    df_titles
    .withColumnRenamed('primaryTitle','nome_filme')
    .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|          nome_filme|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [ ]:
(
    df_titles
    .withColumnRenamed('primaryTitle','nome_filme')
    .select('nome_filme','titleType','startYear','runtimeMinutes')
    .show()
)

+--------------------+---------+---------+--------------+
|          nome_filme|titleType|startYear|runtimeMinutes|
+--------------------+---------+---------+--------------+
|          Carmencita|    short|     1894|             1|
|Le clown et ses c...|    short|     1892|             5|
|      Pauvre Pierrot|    short|     1892|             4|
|         Un bon bock|    short|     1892|            12|
|    Blacksmith Scene|    short|     1893|             1|
|   Chinese Opium Den|    short|     1894|             1|
|Corbett and Court...|    short|     1894|             1|
|Edison Kinetoscop...|    short|     1894|             1|
|          Miss Jerry|    short|     1894|            40|
| Leaving the Factory|    short|     1895|             1|
|Akrobatisches Pot...|    short|     1895|             1|
|The Arrival of a ...|    short|     1896|             1|
|The Photographica...|    short|     1895|             1|
| The Waterer Watered|    short|     1895|             1|
| Autour d'une

In [ ]:
(
    df_titles.selectExpr('primaryTitle as nome_filme', 'titleType', 
                         'startYear','runtimeMinutes').show(5)
)

+--------------------+---------+---------+--------------+
|          nome_filme|titleType|startYear|runtimeMinutes|
+--------------------+---------+---------+--------------+
|          Carmencita|    short|     1894|             1|
|Le clown et ses c...|    short|     1892|             5|
|      Pauvre Pierrot|    short|     1892|             4|
|         Un bon bock|    short|     1892|            12|
|    Blacksmith Scene|    short|     1893|             1|
+--------------------+---------+---------+--------------+
only showing top 5 rows



In [ ]:
df_renamed = df_titles
for c in df_titles.columns : 
  df_renamed = df_renamed.withColumnRenamed(c,c+'_suffx')

df_renamed.limit(5).toPandas()

,tconst_suffx,titleType_suffx,primaryTitle_suffx,originalTitle_suffx,isAdult_suffx,startYear_suffx,endYear_suffx,runtimeMinutes_suffx,genres_suffx
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


**Criando e Alterando Colunas**

Para criar ou alterar colunas é utilizada a função `withColumn()`, da seguinte forma : 
  - `df.withColumn("nome_da_coluna", {expressão geradora da coluna})`

In [ ]:
from pyspark.sql.functions import upper

(
    df_titles
    .select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn("primaryTitle_2",upper('primaryTitle'))
    .show(5)
)

+---------+--------------------+--------------+--------------------+
|   tconst|        primaryTitle|runtimeMinutes|      primaryTitle_2|
+---------+--------------------+--------------+--------------------+
|tt0000001|          Carmencita|             1|          CARMENCITA|
|tt0000002|Le clown et ses c...|             5|LE CLOWN ET SES C...|
|tt0000003|      Pauvre Pierrot|             4|      PAUVRE PIERROT|
|tt0000004|         Un bon bock|            12|         UN BON BOCK|
|tt0000005|    Blacksmith Scene|             1|    BLACKSMITH SCENE|
+---------+--------------------+--------------+--------------------+
only showing top 5 rows



**Criando colunas a partir de constantes**

In [ ]:
from pyspark.sql.functions import lit

(
    df_titles
    .select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('pais',lit('Brasil'))
    .show(5)
)

+---------+--------------------+--------------+------+
|   tconst|        primaryTitle|runtimeMinutes|  pais|
+---------+--------------------+--------------+------+
|tt0000001|          Carmencita|             1|Brasil|
|tt0000002|Le clown et ses c...|             5|Brasil|
|tt0000003|      Pauvre Pierrot|             4|Brasil|
|tt0000004|         Un bon bock|            12|Brasil|
|tt0000005|    Blacksmith Scene|             1|Brasil|
+---------+--------------------+--------------+------+
only showing top 5 rows



**Criando colunas condicionais**

In [ ]:
from pyspark.sql.functions import when

(
    df_titles
    .select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeMinutes',col('runtimeMinutes').cast('int'))
    .withColumn("categoria_runtime", when(col('runtimeMinutes') <=60,'curto')
                                    .when((col('runtimeMinutes') > 60) & (col('runtimeMinutes') < 120),'normal')
                                    .when(col('runtimeMinutes') >=120, 'longo')
                                    .when(col('runtimeMinutes').isNull(), 'nulo')
                                    .otherwise('Erro'))
    .filter('runtimeMinutes > 60')
    .show(25)
)

+---------+--------------------+--------------+-----------------+
|   tconst|        primaryTitle|runtimeMinutes|categoria_runtime|
+---------+--------------------+--------------+-----------------+
|tt0000502|            Bohemios|           100|           normal|
|tt0000574|The Story of the ...|            70|           normal|
|tt0000591|    The Prodigal Son|            90|           normal|
|tt0000679|The Fairylogue an...|           120|            longo|
|tt0001756|Lucha por la here...|            92|           normal|
|tt0002026|Anny - Story of a...|            68|           normal|
|tt0002101|           Cleopatra|           100|           normal|
|tt0002130|     Dante's Inferno|            71|           normal|
|tt0002315|El lobo de la sierra|            76|           normal|
|tt0002423|             Passion|            85|           normal|
|tt0002445|          Quo Vadis?|           120|            longo|
|tt0002452|The Independence ...|           120|            longo|
|tt0002625

In [ ]:
from pyspark.sql.functions import when,expr

predicado = """

CASE WHEN runtimeMinutes <= 60 THEN 'curto'
     WHEN runtimeMinutes > 60 AND runtimeMinutes < 120 THEN 'normal'
     WHEN runtimeMinutes >= 120 THEN 'longo'
     WHEN runtimeMinutes IS NULL THEN 'nulo'
     ELSE 'Erro'
END

"""

(
    df_titles
    .select('tconst','primaryTitle','runtimeMinutes',)
    .withColumn('runtimeMinutes',col('runtimeMinutes').cast('int'))
    .withColumn("categoria_runtime", expr(predicado))
    .filter('runtimeMinutes > 60')
    .show(25)
)

+---------+--------------------+--------------+-----------------+
|   tconst|        primaryTitle|runtimeMinutes|categoria_runtime|
+---------+--------------------+--------------+-----------------+
|tt0000502|            Bohemios|           100|           normal|
|tt0000574|The Story of the ...|            70|           normal|
|tt0000591|    The Prodigal Son|            90|           normal|
|tt0000679|The Fairylogue an...|           120|            longo|
|tt0001756|Lucha por la here...|            92|           normal|
|tt0002026|Anny - Story of a...|            68|           normal|
|tt0002101|           Cleopatra|           100|           normal|
|tt0002130|     Dante's Inferno|            71|           normal|
|tt0002315|El lobo de la sierra|            76|           normal|
|tt0002423|             Passion|            85|           normal|
|tt0002445|          Quo Vadis?|           120|            longo|
|tt0002452|The Independence ...|           120|            longo|
|tt0002625

# **Aula 3.6 -** Trabalhando com diferentes tipos de dados


In [ ]:
import pyspark.sql.functions as f

**Valores Numéricos**

  - `round()` : arredonda o valor numérico
  - `ceil` : arredonda o valor numérico para o maior inteiro mais próximo
  - `floor()` : arredonda o valure numérico para o menor inteiro mais próximo
  - `sqrt()` : retorna a raiz quadrada do valor
  - `exp()` : retorna a exponencial do valor
  - `log()` : retorna o logaritmo natual do valor
  - `log10()` : retorna o logaritmo na base 10 do valor
  - `greatest()` : retorna o maior valor dentre os valores das colunas. Análogo ao `max()`, mas entre colunas. 
  - `least()` : retorna o menor valor dentre os valores das colunas. Análogo ao `min()`, mas entre colunas

In [ ]:
df_titles.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [ ]:
(
    df_titles
    .withColumn('runtimeMinutes',f.col('runtimeMinutes').cast('int'))
    .withColumn('random_normal',f.randn(123))
    .withColumn('dummy_division',f.col('runtimeMinutes')/f.col('random_normal'))
    .withColumn('dummy_division',f.round(f.col('random_normal'),3))
    .withColumn('dummy_round',f.col('runtimeMinutes')*f.col('random_normal'))
    .withColumn('dummy_sum',f.col('runtimeMinutes')+f.col('random_normal'))
    .withColumn('dummy_sub',f.col('runtimeMinutes')-f.col('random_normal'))
    .withColumn('dummy_exp',f.col('runtimeMinutes')**f.col('random_normal'))
    .limit(5)
    .toPandas()
)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,random_normal,dummy_division,dummy_round,dummy_sum,dummy_sub,dummy_exp
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",-0.992755,-0.993,-0.992755,0.007245,1.992755,1.000000
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",0.431839,0.432,2.159195,5.431839,4.568161,2.003745
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",0.250836,0.251,1.003345,4.250836,3.749164,1.415854
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",-0.104760,-0.105,-1.257121,11.895240,12.104760,0.770806
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",-1.301178,-1.301,-1.301178,-0.301178,2.301178,1.000000


**Strings**

  - `upper()` : retorna o string em letras maiúsculas
  - `lower()` : retorna o string em letras minúsculas
  - `initcap()` : retorna a primeira letra de cada palavra no string em letras maiúsculas
  - `trim()` : retira os espaços em branco do início e do final do string
  - `ltrim()/rtrim()` : retira os espaços em branco no início e do final do string, respectivamente
  - `lpad()/rpad()` : acrescenta um carctere no início e no final do srting, respectivamente, até que string tenha um determinado comprimento
  - `length()` : retorna o comprimento do stirng, em quantidade de caracteres
  - `split()` : quebra o string a partir de um padrão e retorna um array com os strings resultantes
  - `concat()` : concatena uma ou mais colunas de string
  - `concat_ws` : concatena uma ou mais colunas de string, com um separador entre elas
  - `regexp_extract()` : retorna um match no string a partir de um padrão regex
  - `regex_replace()` : substitui um match no string a partir de um padrão regex com outros caracteres
  - `substring()` : retornamos caracteres do sting que estão entre os índices especificados. Análogo a f.col().substring() 

In [ ]:
(
    df_titles
    .withColumn('primarytitle',f.upper(f.col('primaryTitle')))
    .withColumn('titleType',f.trim(f.initcap(f.col('titleType'))))
    .withColumn('genres_array',f.split(f.col('genres'),','))
    .withColumn('num_cost',f.substring(f.col('tconst'),3,7))
    .limit(5)
    .toPandas()
)

,tconst,titleType,primarytitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_array,num_cost
0,tt0000001,Short,CARMENCITA,Carmencita,0,1894,\N,1,"Documentary,Short","[Documentary, Short]",0000001
1,tt0000002,Short,LE CLOWN ET SES CHIENS,Le clown et ses chiens,0,1892,\N,5,"Animation,Short","[Animation, Short]",0000002
2,tt0000003,Short,PAUVRE PIERROT,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance","[Animation, Comedy, Romance]",0000003
3,tt0000004,Short,UN BON BOCK,Un bon bock,0,1892,\N,12,"Animation,Short","[Animation, Short]",0000004
4,tt0000005,Short,BLACKSMITH SCENE,Blacksmith Scene,0,1893,\N,1,"Comedy,Short","[Comedy, Short]",0000005


**Datas**

  - `add_months()` : retorna a data depois de adicionar "x" meses
  - `months_between()` : retorna a diferença entre duas datas em meses
  - `date_add()` : retorna a data depois de adicionar "x" dias
  - `date_sub()` : retorna a data depois de subtrair "x" dias
  - `next_day` : retorna o dia seguinte de alguma data
  - `datediff()` : retorna a diferença entre duas datas em dias
  - `current_date` : retorna a data atual
  - `dayofweek/dayofmonth()/dayofyear` : retorna o dia relativo à semana, ao mês e ao ano, respectivamente.
  - `weekofyear` : retorna a semana relativa ao ano
  - `second()/minute()/hour()` : retorna os segundos, os minutos e as horas de uma coluna de date-time, respectivamente
  - `month()/year()` : retorna o mês e o ano de uma coluna de data, respectivamente
  - `last_day()` : retorna o último dia do mês do qual a data considerada pertence
  - `to_date()` : transforma a coluna no tipo data (t.DateType())
  - `trunc()` : formata a data para a unidade especificada
    - `year` : "(ano)-01-01"
    - `month` : "(ano)-(mes)-01" 

In [ ]:
(
    df_titles
    .filter('titleType = "movie"')
    .withColumn('data_ano',f.to_date(f.col('startYear'),'yyyy'))
    .withColumn('mes',f.month(f.col('data_ano')))
    .withColumn('ano',f.dayofmonth(f.col('data_ano')))
    .withColumn('hoje',f.current_date())
    .withColumn('trunc',f.trunc(f.col('data_ano'),'week'))
    .withColumn('ultimo_dia_mes',f.last_day(f.col('data_ano')))
    .withColumn('idade_filme_dias',f.datediff(f.col('hoje'),f.col('data_ano')))
    .withColumn('idade_filme_meses',f.floor(f.months_between(f.col('hoje'),f.col('data_ano'))))
    .withColumn('idade_filme_anos',f.floor(f.col('idade_filme_dias')/365))
    .select('startYear','data_ano','mes','ano','hoje','trunc',
            'ultimo_dia_mes','idade_filme_dias','idade_filme_meses','idade_filme_anos')
    .show(5)
)

+---------+----------+---+---+----------+----------+--------------+----------------+-----------------+----------------+
|startYear|  data_ano|mes|ano|      hoje|     trunc|ultimo_dia_mes|idade_filme_dias|idade_filme_meses|idade_filme_anos|
+---------+----------+---+---+----------+----------+--------------+----------------+-----------------+----------------+
|     1905|1905-01-01|  1|  1|2021-10-29|1904-12-26|    1905-01-31|           42670|             1401|             116|
|     1907|1907-01-01|  1|  1|2021-10-29|1906-12-31|    1907-01-31|           41940|             1377|             114|
|     1907|1907-01-01|  1|  1|2021-10-29|1906-12-31|    1907-01-31|           41940|             1377|             114|
|     1908|1908-01-01|  1|  1|2021-10-29|1907-12-30|    1908-01-31|           41575|             1365|             113|
|     1908|1908-01-01|  1|  1|2021-10-29|1907-12-30|    1908-01-31|           41575|             1365|             113|
+---------+----------+---+---+----------

**Arrays**

  - `array()` : constrói um array com as colunas selecionadas
  - `flatten()` : transforma um array de arrays em um único array
  - `explode()` : retorna uma nova linha para cada elemento no array
  - `size()` : retorna o número de elementos do array
  - `sort_array()` : ordena os elementos do array, de forma crescente ou decrescente
  - `reverse()` : reverte a ordem dos elementos de um array 
  - `array_distinct()` : remove elementos duplicados do array
  - `array_contains()` : verifica se o array contém o elemento especificado
  - `arrays_overlap()` : a partir de 2 colunas de arrays, verifica se elas tem algum elemento em comum, retornando True ou False
  - `array_union()` : a partir de 2 colunas de arrays, retorna o array com os elementos unidos das duas colunas, sem duplicatas
  - `array_except()` : a partir de 2 colunas de arrays, retorna um array com os elementos que estão em uma coluna mais não estão na outra, sem duplicatas
  - `array_intersect()` : a partir de 2 colunas de arrays, retorna um array com os elementos que estão nas duas colunas, sem duplicatas
  - `array_join()` : retorna um string após concaterna os elementos do array usando um delimitador especificado
  - `array_max()/array_min()` : retorna o máximo e o mínimo valor do array, respectivamente
  - `array_remove()` : remove elementos do array que são iguais a uma valor especificado  


In [ ]:
(
    df_titles
    .filter('titleType = "movie"')
    .withColumn('genres_array',f.split(f.col('genres'), ','))
    .withColumn('first_genre',f.col('genres_array')[0])
    .withColumn('genres_string',f.array_join(f.col('genres_array'),','))
    .withColumn('n_genres',f.size(f.col('genres_array')))
    .withColumn('genres_unico',f.explode(f.col('genres_array')))
    .select('genres_array','first_genre','genres_string','n_genres','genres_unico')
    .show(20)
)

+--------------------+-----------+-----------------+--------+------------+
|        genres_array|first_genre|    genres_string|n_genres|genres_unico|
+--------------------+-----------+-----------------+--------+------------+
|                [\N]|         \N|               \N|       1|          \N|
|             [Drama]|      Drama|            Drama|       1|       Drama|
|             [Drama]|      Drama|            Drama|       1|       Drama|
|             [Drama]|      Drama|            Drama|       1|       Drama|
|             [Drama]|      Drama|            Drama|       1|       Drama|
|[Adventure, Fantasy]|  Adventure|Adventure,Fantasy|       2|   Adventure|
|[Adventure, Fantasy]|  Adventure|Adventure,Fantasy|       2|     Fantasy|
|             [Drama]|      Drama|            Drama|       1|       Drama|
|             [Drama]|      Drama|            Drama|       1|       Drama|
|                [\N]|         \N|               \N|       1|          \N|
|                [\N]|   

**Nulos**

  - `drop()` : retira do DataFrame as linhas com nulos, com base no que foi passado para o argumento how
    - any (default) : retira todas as linhas com pelo menos um valor nulo nas colunas
    - all : somente retira as linhas com todos os valores nulos nas colunas
  - `fill()` : preenche os valores nulos no DataFrame com uma constante, passada pelo usuário
  - `replace()` : substitui o valor (não somente os valores nulos) por algum outro passado pelo usuário  

In [ ]:
(
    df_titles
    .replace('\\N', None, subset=['startYear','endYear'])
    .filter('endYear is null and startYear is not null')
    #.na.drop(subset=['startYear'])#'0',subset=['endYear'])
    .withColumn('coalesce_test',f.coalesce(f.col('startYear'),
                                           f.col('startYear'),
                                           f.lit('Sem ano')))
    .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|coalesce_test|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|   null|             1|   Documentary,Short|         1894|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|   null|             5|     Animation,Short|         1892|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|   null|             4|Animation,Comedy,...|         1892|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|   null|            12|     Animation,Short|         1892|
|tt0000005|    short

# **Aula 4.1 -** Manipulando dados com Spark - Parte II